# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [48]:
from typing import Tuple, List
import math, time

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

Wielkość planszy

In [49]:
ROW_COUNT = 6
COLUMN_COUNT = 7
WINDOW_LENGTH = 4
CURRENT_PLAYER = None

In [50]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour, depth, alpha_beta_on):
        self.game = game
        self.depth = depth
        self.alpha_beta_on = alpha_beta_on

    def evaluate_position(self, player: Player)->float:
        score = 0
        temp_field = self.game.state.fields
        # Score horizontal positions
        for r in range(ROW_COUNT):
            row_array = [temp_field[i][r] for i in range(COLUMN_COUNT)]
            #for some weirds reasons column and row indexes are switched in game.state.fields
            for c in range(COLUMN_COUNT - 3):
                window_temp = row_array[c:c + WINDOW_LENGTH]
                window = [elem.char if elem is not None else None for elem in window_temp]
                score += self.evaluate_window(window, player)
        # Score vertical positions
        for c in range(COLUMN_COUNT):
            col_array = [temp_field[c][i] for i in range(ROW_COUNT)]
            #for some weirds reasons column and row indexes are switched in game.state.fields
            for r in range(ROW_COUNT - 3):
                window_temp = col_array[r:r + WINDOW_LENGTH]
                window = [elem.char if elem is not None else None for elem in window_temp]
                score += self.evaluate_window(window, player)
        # Score positive diagonals
        for r in range(ROW_COUNT - 3):
            for c in range(COLUMN_COUNT - 3):
                window_temp = [temp_field[c + i][r + i] for i in range(WINDOW_LENGTH)]
                #for some weirds reasons column and row indexes are switched in game.state.fields
                window = [elem.char if elem is not None else None for elem in window_temp]
                score += self.evaluate_window(window, player) * 0.8
        # Score negative diagonals
        for r in range(ROW_COUNT - 3):
            for c in range(COLUMN_COUNT - 3):
                # Create a negative diagonal window of 4
                window_temp = [temp_field[c + i][r + 3 - i] for i in range(WINDOW_LENGTH)]
                #for some weirds reasons column and row indexes are switched in game.state.fields
                window = [elem.char if elem is not None else None for elem in window_temp]
                score += self.evaluate_window(window, player) * 0.8
        return score

    def evaluate_window(self, window, player: Player)->float:
        score = 0
        if player.char == self.game.first_player.char:
            current_player = self.game.first_player
            opposite_player = self.game.second_player
        else:
            current_player = self.game.second_player
            opposite_player = self.game.first_player

        if window.count(current_player.char) == 4:
            score += 100
        elif window.count(current_player.char) == 3 and window.count(None) == 1:
            score += 13
        elif window.count(current_player.char) == 2 and window.count(None) == 2:
            score += 3
        elif window.count(current_player.char) == 1 and window.count(None) == 3:
            score += 1

        if window.count(opposite_player.char) == 4:
            score -= 100
        elif window.count(opposite_player.char) == 3 and window.count(None) == 1:
            score -= 5
        elif window.count(opposite_player.char) == 2 and window.count(None) == 2:
            score -= 1
        return score

    def get_best_move(self)->int:
        pass

    def is_valid_move(self, col_index:int)->bool:
        return self.game.state.fields[col_index][ROW_COUNT-1] == None

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        is_finished = self.game.is_finished()

        if depth == 0 or is_finished:
            if is_finished:
                if self.game.get_winner() is not None:
                    if self.game.get_winner().char == CURRENT_PLAYER.char:
                        return None, 100000
                    else:
                        return None, -100000
                else:
                    return None, 0
            else: # Depth is zero
                return None, self.evaluate_position(CURRENT_PLAYER)
        if is_maximizing_player:
            value = -math.inf
            for col in range(COLUMN_COUNT):
                if self.is_valid_move(col):
                    if depth == self.depth:
                        if CURRENT_PLAYER.char == 'a':
                            game_copy = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.first_player, second_player=self.game.second_player)
                        else:
                            game_copy = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.second_player, second_player=self.game.first_player)
                    else:
                        game_copy = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.second_player, second_player=self.game.first_player)

                    game_copy.state.fields = self.game.state.fields
                    game_copy.make_move(ConnectFourMove(col)) # Gotta check if _current_player is valid
                    copy_minimax = MinMaxSolver(game_copy, self.depth, self.alpha_beta_on)
                    new_score = copy_minimax.minimax(depth-1, alpha, beta, False)[1]

                    if new_score > value:
                        value = new_score
                        column = col
                    if self.alpha_beta_on:
                        alpha = max(alpha, value)
                        if alpha >= beta:
                            break
        else: # Minimizing player
            value = math.inf
            for col in range(COLUMN_COUNT):
                if self.is_valid_move(col):
                    game_copy = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=self.game.second_player, second_player=self.game.first_player)
                    game_copy.state.fields = self.game.state.fields
                    game_copy.make_move(ConnectFourMove(col)) # Gotta check if _current_player is valid
                    copy_minimax = MinMaxSolver(game_copy, self.depth, self.alpha_beta_on)
                    new_score = copy_minimax.minimax(depth-1, alpha, beta, True)[1]

                    if new_score < value:
                        value = new_score
                        column = col
                    if self.alpha_beta_on:
                        beta = min(beta, value)
                        if alpha >= beta:
                            break

        return column, value


Rozgrywka

In [51]:


depths = [2, 3, 5, 6]
times = []
for depth in depths:
    for mode in range(2):
        if mode == 0:
            alpha_beta_on = False
        else:
            alpha_beta_on = True
        p1 = Player("a")
        p2 = Player("b")
        game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
        mini_max_solver = MinMaxSolver(game, depth, alpha_beta_on)
        iteration = 0
        start_time = time.time()
        print(f'\tDepth: {depth}, Alpha-beta pruning mode: {alpha_beta_on}\n')
        while not game.is_finished():
            CURRENT_PLAYER = game.get_current_player()
            column, value = mini_max_solver.minimax(depth, -math.inf, math.inf, True)
            game.make_move(ConnectFourMove(column))
            iteration += 1
            print(iteration)
            print(game)
        times.append(time.time() - start_time)
        if game.get_winner() is not None:
            print(f'Winner: {game.get_winner().char}')

for i in range(len(depths)):
    info = f'\tDepth: {depths[i]}, '
    print(info + f'\nExecution times: No pruning: {times[2*i]}; Pruning: {times[2*i+1]}')

	Depth: 2, Alpha-beta pruning mode: False

1
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
2
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
3
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
4
Current player: a
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
5
Current player: b
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
6
Current player: a
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
[ ][ ][ ][b][ ][ ][ ]
[ ][ ][ ][a][ ][ ][ ]
7
Current player: b
[ ][ ][ ][b][ ][ ][ ]
[ ]

Wnioski:

Metoda MiniMax wydaje się efektywnie szukać optymalnych ruchów dla zadanej głębokości przeszukiwania.
Sednem gwarancji jakości solwera wydaje się heurystyka, która fundamentalnie determinuje wybór ruchów. Szczególnym problemem jest ocena okien na planszy (czterech kolejnych pól), która znajdują się na pewnej wysokości tj. przeszacowywanie okien, które realnie osiągniętę mogą być dopiero za relatywnie wiele ruchów. Ostatecznie nie podjąłem się uwzględnienia tej wysokości w heurystyce. Niemniej solwer wydaje się działać relatywnie dobrze, w tym zauważa stany terminalne i na nie reaguje. Warto zaznaczyć jak kluczowe jest odcinanie alfa-beta, które diametralnie skraca czas wykonania solwera.
W teorii odcinanie to powinno skrócić czas przeszukiwania średnio o 50% (tak podaje literatura), lecz jak widać w tym przypadku oszczędność ta jest tym większa, czym większa jest głębokość przeszukiwania.

Conclusions:

The MiniMax method appears to effectively search for optimal moves for a given depth of exploration. The core of the solver's quality assurance seems to be the heuristic, which fundamentally determines the choice of moves. A particular challenge is the evaluation of windows on the board (four consecutive fields) that are at a certain height, i.e., estimating windows that can realistically be reached only after a relatively large number of moves. Ultimately, I did not incorporate this height consideration into the heuristic. Nevertheless, the solver seems to work relatively well, detecting terminal states and reacting to them. It is worth noting how crucial alpha-beta pruning is, as it drastically reduces the solver's execution time. In theory, this pruning should reduce the search time by an average of 50% (as stated in the literature), but as evident in this case, the savings are greater the deeper the exploration depth.